Importazione delle librerie utili

In [1]:
import sklearn
import pandas
import numpy
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree

Lettura dei dati

In [2]:
#leggo i dati di training
f1 = pandas.read_csv('train.csv')
#leggo i dati di test
f2 = pandas.read_csv('test.csv')

Conversione e analisi dei dati sul training set

In [3]:
#separo i dati dal "risultato" per quanto riguarda i dati di training
dati_training_stringa = f1.drop(['status'], axis = 1)
target_training = f1['status']

#separo i dati dal "risultato" per quanto riguarda i dati di test
dati_test_stringa = f2.drop(['status'], axis = 1)
target_test = f2['status']

In [4]:
#Utilizzo gli One Hot Encoder per convertire tutti i dati in float
dati_training = dati_training_stringa
dati_training = pandas.concat([dati_training, pandas.get_dummies(dati_training['gender'], prefix = 'gender')], axis = 1)
dati_training = pandas.concat([dati_training, pandas.get_dummies(dati_training['ssc_b'], prefix = 'ssc_b')], axis = 1)
dati_training = pandas.concat([dati_training, pandas.get_dummies(dati_training['hsc_b'], prefix = 'hsc_b')], axis = 1)
dati_training = pandas.concat([dati_training, pandas.get_dummies(dati_training['hsc_s'], prefix = 'hsc_s')], axis = 1)
dati_training = pandas.concat([dati_training, pandas.get_dummies(dati_training['degree_t'], prefix = 'degree_t')], axis = 1)
dati_training = pandas.concat([dati_training, pandas.get_dummies(dati_training['workex'], prefix = 'workex')], axis = 1)
dati_training = pandas.concat([dati_training, pandas.get_dummies(dati_training['specialisation'], prefix = 'specialisation')], axis = 1)

dati_training.drop(['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation'], axis = 1, inplace = True)

In [5]:
#Elimino le colonne con dipendenze verso altre colonne
#Ho deciso di eliminare anche la feature salary perchè contiene più del 30% di valori nulli, per cui uno split su tale feature
#portava il modello ad avere un'accuracy piuttosto basta sul test set
dati_training = dati_training.drop(['gender_F'], axis = 1)
dati_training = dati_training.drop(['ssc_b_Central'], axis = 1)
dati_training = dati_training.drop(['hsc_b_Others'], axis = 1)
dati_training = dati_training.drop(['workex_No'], axis = 1)
dati_training = dati_training.drop(['specialisation_Mkt&Fin'], axis = 1)
dati_training = dati_training.drop(['hsc_s_Commerce'], axis = 1)
dati_training = dati_training.drop(['degree_t_Comm&Mgmt'], axis = 1)
dati_training = dati_training.drop(['salary'], axis = 1)

In [10]:
#Stampa del grafico che indica la correlazione tra i dati di training
corr_df = dati_training.corr(method='pearson')
plt.figure(figsize=(20, 18))
sns.heatmap(corr_df, annot=True)
plt.show()

Conversione dei dati sul test set

In [7]:
dati_test = dati_test_stringa
dati_test = pandas.concat([dati_test, pandas.get_dummies(dati_test['gender'], prefix = 'gender')], axis = 1)
dati_test = pandas.concat([dati_test, pandas.get_dummies(dati_test['ssc_b'], prefix = 'ssc_b')], axis = 1)
dati_test = pandas.concat([dati_test, pandas.get_dummies(dati_test['hsc_b'], prefix = 'hsc_b')], axis = 1)
dati_test = pandas.concat([dati_test, pandas.get_dummies(dati_test['hsc_s'], prefix = 'hsc_s')], axis = 1)
dati_test = pandas.concat([dati_test, pandas.get_dummies(dati_test['degree_t'], prefix = 'degree_t')], axis = 1)
dati_test = pandas.concat([dati_test, pandas.get_dummies(dati_test['workex'], prefix = 'workex')], axis = 1)
dati_test = pandas.concat([dati_test, pandas.get_dummies(dati_test['specialisation'], prefix = 'specialisation')], axis = 1)

dati_test.drop(['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation'], axis = 1, inplace = True)

dati_test = dati_test.drop(['gender_F'], axis = 1)
dati_test = dati_test.drop(['ssc_b_Central'], axis = 1)
dati_test = dati_test.drop(['hsc_b_Others'], axis = 1)
dati_test = dati_test.drop(['workex_No'], axis = 1)
dati_test = dati_test.drop(['specialisation_Mkt&Fin'], axis = 1)
dati_test = dati_test.drop(['hsc_s_Commerce'], axis = 1)
dati_test = dati_test.drop(['degree_t_Comm&Mgmt'], axis = 1)
dati_test = dati_test.drop(['salary'], axis = 1)

Addestramento modello ed emissione dell'accuracy

In [15]:
albero = DecisionTreeClassifier()
albero.fit(dati_training, target_training)
print("Accuracy on training set: {:.3f}".format(albero.score(dati_training, target_training)))
print("Accuracy on test set: {:.3f}".format(albero.score(dati_test, target_test)))

Analisi del modello

In [12]:
#Stampo l'albero di decisione ottenuto utilizzando plot_tree da sklearn, ma avrei anche potuto utilizzare anche graphviz come visto a lezione
fn = ['ssc_p','hsc_p','degree_p','etest_p', 'mba_p', 'gender_M', 'ssc_b_Others', 'hsc_b_Central', 'hsc_s_Arts', 'hsc_s_Science', 'degree_t_Others', 'degree_t_Sci&Tech', 'workex_Yes', 'specialisation_Mkt&HR']
cn = ['Not Placed', 'Placed']
plt.figure(figsize=(20,20))
plot_tree(albero, feature_names = fn, class_names = cn);

In [13]:
#Verifico le feature importances
print("Feature importances:")
print(albero.feature_importances_)

In [12]:
#Stampo un grafico che rappresenta le feature importances in modo più esplicito
plt.barh(numpy.arange(albero.n_features_), albero.feature_importances_, align='center')
plt.yticks(numpy.arange(albero.n_features_), fn)
plt.xlabel("Feature importance")
plt.ylabel("Feature")
plt.ylim(-1, albero.n_features_)

#Tale grafico mostra su quali features sono eseguiti gli split, quindi si potrebbero eliminare le altre